In [1]:
!pip install AugmentedSocialScientist pandas numpy wordcloud matplotlib spacy nltk
!python -m spacy download fr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 9.4 MB/s eta 0:00:00:00:010:01m
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')


REMARQUE : BESOIN D'AVOIR FAIT TOURNE DESCRIPTION ANNOTATIONS POUR CREER LE FICHIER AVEC LES PHRASES !

In [2]:
import numpy as np
import random
import torch
import sklearn
random_seed = 1968
# Set random seed for NumPy
np.random.seed(random_seed)

# Set random seed for Python built-in random module
random.seed(random_seed)

# Set random seed for PyTorch
torch.manual_seed(random_seed)

# Set random seed for Scikit-learn
sklearn.utils.check_random_state(random_seed)


RandomState(MT19937) at 0x7F3151AA9940

In [3]:
import pandas as pd

current_annotations_file = "../../data/intermediate/annotations/personne_withSentence_1400annotations_2024-06-03_ActiveTigger.csv"

annotations_files = pd.read_csv(current_annotations_file, dtype = {'sentence_id':'int', 
'Personne':'str','sentence':'str'}).set_index('sentence_id')

recat = {
    "Corps humain":"Corps humain",
    "Genre/espèce humaine":"Genre/espèce humaine",
    "Individu ou sujet" : "Individu ou sujet",
    "Personne à protéger":"Personne droits et devoirs",
    "Personne responsable et autonome":"Personne droits et devoirs",
    "Personne relationnelle":"Personne relationnelle",
    "Sentiment humain":"Genre/espèce humaine",
    "Poubelle" : "Poubelle"
}

frecat = lambda x: recat[x]
annotations_files['recat']=annotations_files["Personne"].apply(frecat)


annotations = annotations_files[~(annotations_files["recat"]=="Poubelle")]

annotations.sample(5)

,Personne,sentence,recat
sentence_id,,,
1190093,Personne responsable et autonome,"En revanche, \nleur rapidité et leur relative ...",Personne droits et devoirs
870236,Personne relationnelle,Refus par un tiers 8 \n \nIl est de nombreuses...,Personne relationnelle
1210148,Sentiment humain,"Elle est une réalité \nhumaine, qui tient à la...",Genre/espèce humaine
860089,Personne responsable et autonome,"Ainsi l’ensemble de ces inconvénients qui, pou...",Personne droits et devoirs
1130224,Corps humain,21415 »30 Voir l’avis n°112 du CCNE « Une réf...,Corps humain


In [4]:
len(annotations)

1388

# Bert classification binaire

In [5]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

from torch import cuda

cuda.get_device_name(0)

'NVIDIA A2'

In [6]:
from sklearn.model_selection import train_test_split
 
train_data, test_data = train_test_split(annotations, train_size=0.80, random_state=random_seed, stratify=annotations["Personne"])

In [7]:
from AugmentedSocialScientist.models import Camembert

bert = Camembert()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


There are 1 GPU(s) available.
We will use GPU 0: NVIDIA A2


In [8]:
train_data['text'] = train_data['sentence']
test_data['text'] = test_data['sentence']

scores_by_cat = dict()
categories_for_models = ["Corps humain","Genre/espèce humaine","Individu ou sujet",
            "Personne relationnelle","Personne droits et devoirs"]
for cat in categories_for_models:
    train_data['label'] = (train_data['recat']==cat)
    test_data['label'] = (test_data['recat']==cat)
    print(cat)
    batch_size = 16
    train_loader = bert.encode(train_data.text.values,
    train_data.label.values, batch_size=batch_size)

    test_loader = bert.encode(test_data.text.values,
    test_data.label.values, batch_size=batch_size)

    scores = bert.run_training(
        train_loader,
        test_loader,
        n_epochs=5,
        lr=2e-5,
        random_state=random_seed,
        save_model_as=cat
        )

    scores_by_cat[cat] = scores


Corps humain


  0%|          | 0/1110 [00:00<?, ?it/s]

  0%|          | 0/1110 [00:00<?, ?it/s]

label ids: {False: 0, True: 1}


  0%|          | 0/278 [00:00<?, ?it/s]

...

  0%|          | 0/278 [00:00<?, ?it/s]

  0%|          | 0/278 [00:00<?, ?it/s]

label ids: {False: 0, True: 1}


Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/mamba/lib/python3.12/site-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(



======== Epoch 1 / 5 ========
Training...
  Batch    40  of     70.    Elapsed: 0:00:49.

  Average training loss: 0.61
  Training took: 0:01:25

Running Validation...

  Average test loss: 0.53
  Validation took: 0:00:07
              precision    recall  f1-score   support

       False       0.72      0.99      0.83       194
        True       0.82      0.11      0.19        84

    accuracy                           0.72       278
   macro avg       0.77      0.55      0.51       278
weighted avg       0.75      0.72      0.64       278


======== Epoch 2 / 5 ========
Training...
  Batch    40  of     70.    Elapsed: 0:00:49.

  Average training loss: 0.47
  Training took: 0:01:25

Running Validation...

  Average test loss: 0.45
  Validation took: 0:00:07
              precision    recall  f1-score   support

       False       0.86      0.83      0.85       194
        True       0.64      0.69      0.66        84

    accuracy                           0.79       278
   macro 

In [15]:
for cat in categories_for_models:
    scores = scores_by_cat[cat]
    categories = ['False','True']
    pd.DataFrame({
        "category" : categories,
        "precision" : scores[0],
        "recall" : scores[1],
        "f1-score" : scores[2],
        "support" : scores[3]
    }).set_index("category").to_csv(f"./models/{cat}/scores.csv")
    print(cat)
    print(
    pd.DataFrame({
        "category" : categories,
        "precision" : scores[0],
        "recall" : scores[1],
        "f1-score" : scores[2],
        "support" : scores[3]
    }).set_index("category")
    )

Corps humain
          precision    recall  f1-score  support
category                                        
False      0.969298  0.960870  0.965066      230
True       0.820000  0.854167  0.836735       48
Genre/espèce humaine
          precision    recall  f1-score  support
category                                        
False      0.941704  0.925110  0.933333      227
True       0.690909  0.745098  0.716981       51
Individu ou sujet
          precision    recall  f1-score  support
category                                        
False      0.895833  0.942982  0.918803      228
True       0.657895  0.500000  0.568182       50
Personne relationnelle
          precision    recall  f1-score  support
category                                        
False      0.929167  0.957082  0.942918      233
True       0.736842  0.622222  0.674699       45
Personne droits et devoirs
          precision    recall  f1-score  support
category                                        
False      0.855

In [16]:
import os
import zipfile

def zip_folder(folder_path, zip_file_path):
    with zipfile.ZipFile(zip_file_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                # Create a relative path for the file inside the zip archive
                file_path = os.path.join(root, file)
                relative_path = os.path.relpath(file_path, folder_path)
                zipf.write(file_path, relative_path)
    print(f"Folder '{folder_path}' successfully zipped to '{zip_file_path}'")

# Example usage
folder_path = 'models'
zip_file_path = 'models.zip'
zip_folder(folder_path, zip_file_path)


Folder 'models' successfully zipped to 'models.zip'
